In [ ]:
!pip install contractions

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import contractions               #Won't -> will not
import re                         #Regex
import nltk                       #Tokenizar texto
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

stopwords_en = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
tqdm.pandas()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/1252/Procesamiento de Lenguaje Natural/Archivos/Articles.csv', encoding = "latin-1")

In [ ]:
df.head()

In [ ]:
df['NewsType'].unique()

In [ ]:
embeddings = {}

with open('/content/drive/MyDrive/1252/Procesamiento de Lenguaje Natural/Archivos/glove.6B/glove.6B.300d.txt', encoding='utf-8') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vectors = np.asarray(values[1:])
    embeddings[word] = vectors

In [ ]:
def vectorize(texto):
  vector_size = 300 #Dependiento del archivo de glove -> glove.6B.300d.txt

  #Todo a minusculas
  texto = texto.lower()

  #Contracciones
  texto = contractions.fix(texto)

  #Regex para quitar todo lo que no sea letra
  texto = re.sub(r'[^a-z\s]','', texto)

  #Quitar stop words
  texto = word_tokenize(texto)

  #Eliminar stopwords
  texto = [token for token in texto if token not in stopwords_en]

  #Lemmatizar
  texto = [lemmatizer.lemmatize(word) for word in texto]

  #Generar vector (sumar linealmente)
  vector = np.zeros(vector_size)

  for word in texto:
    if word in embeddings:
      vector += embeddings[word].astype(float)

  vector = vector.reshape(1, -1)[0] #Una fila, columans necesarias

  return vector

In [ ]:
vectorize('This is a test')

In [ ]:
df['vector'] = df['Article'].progress_apply(vectorize)

In [ ]:
from sklearn.model_selection import train_test_split
